In [1]:
import os
from flask import Flask, request
from werkzeug.utils import secure_filename

app = Flask(__name__)

# 定义上传文件夹路径
UPLOAD_FOLDER = os.path.join(app.root_path, 'uploads')
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# 获取 instance 文件夹中 site.db 的路径
db_path = os.path.join(os.path.dirname(os.path.dirname(app.root_path)), 'instance', 'site.db')
print(f"数据库文件的完整路径: {db_path}")

import sqlite3

# 连接到数据库
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
# 创建新表（修改表结构以存储文件路径）
cursor.execute('''
CREATE TABLE IF NOT EXISTS applications (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    app_name TEXT NOT NULL,
    app_avatar_path TEXT,
    creator_name TEXT NOT NULL,
    creator_avatar_path TEXT,
    app_description TEXT,
    doc_file_path TEXT,
    visit_count INTEGER DEFAULT 0,
    comment_count INTEGER DEFAULT 0,
    favorite_count INTEGER DEFAULT 0,
    used_models TEXT
)
''')

# 提交更改并关闭连接
conn.commit()
conn.close()

print("表 'applications' 已成功创建。")

# 文件上传函数
def allowed_file(filename):
    ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif', 'doc', 'docx'}
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return 'No file part', 400
    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        return f'File uploaded successfully. Path: {file_path}', 200
    return 'File type not allowed', 400

# 验证表是否创建成功
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 查询所有表
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("数据库中的所有表:")
for table in tables:
    print(table[0])

conn.close()

数据库文件的完整路径: d:\2024秋招-大四上\毕业论文\instance\site.db
表 'applications' 已成功创建。
数据库中的所有表:
user
question
applications
sqlite_sequence
